### Install PyTorch
pip3 install torch torchvision torchaudio
pip install Accelerate

### Install Transformers
pip install transformers

### Install Whisper
pip install -U openai-whisper

### Intall ffmpeg
- on Ubuntu or Debian
sudo apt update && sudo apt install ffmpeg

- on MacOS using Homebrew (https://brew.sh/)
brew install ffmpeg

### Install rust in case tiktoken does not provide a pre-built wheel for platform
pip install setuptools-rust

### Install progress bar
pip install progress_bar

In [15]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [16]:
has_mps = torch.backends.mps.is_available()
has_cuda = torch.cuda.is_available()
device = torch.device("cuda" if has_cuda else "cpu")
print(f"Using {'Metal' if has_mps else 'CUDA'} with device {device}")
torch_dtype = torch.float16 if has_mps else torch.float32
device, torch_dtype

Using Metal with device cpu


(device(type='cpu'), torch.float16)

In [17]:
# load Whistler model
model_id = "openai/whisper-large-v3"
#model_id = "openai/whisper-medium"

hf_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    cache_dir="/users/yau/.cache/whisper/models"
).to(device=device)
processor = AutoProcessor.from_pretrained(model_id)


In [18]:
pipe = pipeline(
    task="automatic-speech-recognition",
    model=hf_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    #max_new_tokens=128,
    #chunk_length=64,
    batch_size=24,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
    #ignore_warnings=True,
)

Device set to use cpu


In [2]:
from pydub import AudioSegment

audio_file = "5760-Nano-L2.mp3"
sound = AudioSegment.from_file(audio_file)
print(f"Duration: {round(len(sound) / 1000 /60, 2)} minutes")

excepted_tran_time = len(sound) / 1000 / 60 / 20
print(f"Expected transcription time: {round(excepted_tran_time, 2)} minutes")


Duration: 156.75 minutes
Expected transcription time: 7.84 minutes


In [3]:
import progress_bar
@progress_bar(except_time=excepted_tran_time)
def transcribe(a):
    sound = AudioSegment.from_file(audio_file)
    return pipe(sound)


ImportError: cannot import name 'IncrementalBar' from 'progress_bar' (/opt/anaconda3/envs/transcribe311/lib/python3.11/site-packages/progress_bar.py)